In [1]:
import sys
from pathlib import Path
import warnings
warnings.filterwarnings("ignore", module="IPython")

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('aurora',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir) 
    print("Local environment")

print(f"Root dir: {root_dir}")

# Add the root directory to the `PYTHONPATH` 
if root_dir not in sys.path:
    sys.path.append(root_dir)
    print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Root dir: C:\Users\lppap\Documents\master\scalable_ML\id2223-project
Added the following directory to the PYTHONPATH: C:\Users\lppap\Documents\master\scalable_ML\id2223-project
HopsworksSettings initialized!


## Imports

In [2]:
import os
from datetime import datetime, timedelta
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix
)
import hopsworks
from mlfs.aurora import util
import json

import warnings
warnings.filterwarnings("ignore")

FEATURE_VIEW_NAME = "aurora_fv_final"
FEATURE_VIEW_VERSION = 1   # change to 2 for solar wind, 1 no solar wind
MODEL_NAME = "aurora_xgboost"


HORIZONS = [1, 2, 3, 4, 5]

RANDOM_STATE = 42
TEST_RATIO = 0.2

## Hopsworks login & Connect

In [3]:
project = hopsworks.login(engine="python")
fs = project.get_feature_store() 

2026-01-05 12:14:18,628 INFO: Initializing external client
2026-01-05 12:14:18,628 INFO: Base URL: https://c.app.hopsworks.ai:443


2026-01-05 12:14:20,439 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1279154


In [4]:
# Retrieve feature groups
geomagnetic_fg = fs.get_feature_group( name="geomagnetic_daily_final", version=1)
weather_fg = fs.get_feature_group(name="sweden_weather_daily_final", version=1)

## Feature View creation

In [5]:
# Select and join features for training
query = (
    geomagnetic_fg.select(
        ["date", "kp1", "kp2", "kp3", "kp4", "kp5", "kp6", "kp7", "kp8", "ap1", "ap2", "ap3", "ap4", "ap5", "ap6", "ap7", "ap8", "ap"]
    ).join(
        weather_fg.select([
            "cloud_cover_mean",
            "precipitation_sum",
            "sunshine_duration"
        ]),
        on=["date"]
    ))

## Create the feature view

In [6]:
try:
    feature_view = fs.get_or_create_feature_view(
        name=FEATURE_VIEW_NAME,
        description="Geomagnetic and weather features for aurora visibility prediction",
        version=FEATURE_VIEW_VERSION,
        labels=["ap"],
        query=query,
    )
except Exception:
    feature_view = fs.get_feature_view(
        name=FEATURE_VIEW_NAME,
        version=FEATURE_VIEW_VERSION
    )

In [7]:
'''
feature_view_old = fs.get_or_create_feature_view(
    name=FEATURE_VIEW_NAME,
    description="Geomagnetic and weather features for aurora visibility prediction",
    version=FEATURE_VIEW_VERSION,
    labels=["ap"],
    query=query,
)
'''

'\nfeature_view_old = fs.get_or_create_feature_view(\n    name=FEATURE_VIEW_NAME,\n    description="Geomagnetic and weather features for aurora visibility prediction",\n    version=FEATURE_VIEW_VERSION,\n    labels=["ap"],\n    query=query,\n)\n'

In [8]:
'''
feature_view = fs.get_feature_view(
    name=FEATURE_VIEW_NAME,
    version=FEATURE_VIEW_VERSION
)
'''

'\nfeature_view = fs.get_feature_view(\n    name=FEATURE_VIEW_NAME,\n    version=FEATURE_VIEW_VERSION\n)\n'

In [9]:
X_base, y_base = feature_view.training_data(
    description="Training data for multi-horizon aurora models",
    training_dataset_version=1
)

X_base["date"] = pd.to_datetime(X_base["date"], utc=True)

print("Date range in training data:")
print(X_base["date"].min(), "→", X_base["date"].max())

print(X_base.shape)
print(X_base.columns)
X_base.head()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (2.22s) 
Date range in training data:
2020-01-01 00:00:00+00:00 → 2026-01-04 00:00:00+00:00
(2191, 20)
Index(['date', 'kp1', 'kp2', 'kp3', 'kp4', 'kp5', 'kp6', 'kp7', 'kp8', 'ap1',
       'ap2', 'ap3', 'ap4', 'ap5', 'ap6', 'ap7', 'ap8', 'cloud_cover_mean',
       'precipitation_sum', 'sunshine_duration'],
      dtype='object')


,date,kp1,kp2,kp3,kp4,kp5,kp6,kp7,kp8,ap1,ap2,ap3,ap4,ap5,ap6,ap7,ap8,cloud_cover_mean,precipitation_sum,sunshine_duration
0,2020-02-17 00:00:00+00:00,1.333,0.333,1.333,0.667,1.000,1.333,3.000,3.000,5.0,2.0,5.0,3.0,4.0,5.0,15.0,15.0,80.333336,1.1,14275.975586
1,2020-03-15 00:00:00+00:00,0.000,0.333,0.333,1.667,1.333,1.000,1.667,1.667,0.0,2.0,2.0,6.0,5.0,4.0,6.0,6.0,88.583336,1.7,469.042084
2,2020-04-06 00:00:00+00:00,1.000,1.000,0.333,0.000,1.000,0.333,0.000,0.000,4.0,4.0,2.0,0.0,4.0,2.0,0.0,0.0,67.708336,0.0,39656.863281
3,2020-04-10 00:00:00+00:00,0.000,0.667,1.000,1.333,1.000,2.333,1.000,0.333,0.0,3.0,4.0,5.0,4.0,9.0,4.0,2.0,67.166664,0.6,44831.945312
4,2020-04-15 00:00:00+00:00,3.000,2.667,1.667,1.000,1.000,0.667,1.000,1.333,15.0,12.0,6.0,4.0,4.0,3.0,4.0,5.0,75.250000,0.8,19225.437500


## Feature engineering - Lagged geomagnetic features

In [12]:
# Ensure data is sorted by time
order = X_base["date"].sort_values().index
X_base = X_base.loc[order].reset_index(drop=True)
y_base = y_base.loc[order].reset_index(drop=True)

ap = y_base["ap"]

# Lagged Ap features
#for lag in [1, 2, 3]:
#    X_base[f"ap_lag_{lag}"] = ap.shift(lag)
    

# Lagged Kp features (daily mean + max are most informative)
kp_cols = [f"kp{i}" for i in range(1, 9)]
X_base["kp_mean"] = X_base[kp_cols].mean(axis=1)
X_base["kp_max"] = X_base[kp_cols].max(axis=1)

for lag in [1, 2, 3]:
    X_base[f"kp_mean_lag_{lag}"] = X_base["kp_mean"].shift(lag)
    X_base[f"kp_max_lag_{lag}"] = X_base["kp_max"].shift(lag)

In [13]:
# Build binary targets for all horizons
AP_THRESHOLD = 15
MAX_HORIZON = 5

y_targets = {
    h: (ap.shift(-h) >= AP_THRESHOLD).astype("int32")
    for h in range(1, MAX_HORIZON + 1)
}

X = X_base.drop(columns=["date"])

### Training loop — one model per horizon

In [14]:
# Train / test split (time-aware,
split_idx = int((1 - TEST_RATIO) * len(X))

X_train = X.iloc[:split_idx]
X_test  = X.iloc[split_idx:]

train_mask = X_train.index
test_mask  = X_test.index

## 5 days target models

Design rules:

- One horizon = one model
- Same features for all horizons
- Only the target shifts
- Same train/test split for all horizons
- One model name per horizon 

In [15]:
import pandas as pd
import seaborn as sns

def feature_importance_plot(model, model_dir):
    # Get feature importance by gain
    importance = model.get_booster().get_score(importance_type="gain")
    
    # Convert to DataFrame
    imp_df = (
        pd.DataFrame(importance.items(), columns=["feature", "gain"])
          .sort_values("gain", ascending=False)
          .head(20)
    )
    
    # Plot
    plt.figure(figsize=(6, 4))
    sns.barplot(
        data=imp_df,
        x="gain",
        y="feature",
        color="steelblue"
    )
    plt.title(f"Top 20 Feature Importances (t+{h})")
    plt.xlabel("Gain")
    plt.ylabel("Feature")
    
    plt.tight_layout()
    plt.savefig(f"{model_dir}/feature_importance_gain_tplus{h}.png")
    plt.close()

In [16]:
from xgboost import XGBClassifier
from sklearn.metrics import (
    roc_auc_score, accuracy_score,
    precision_score, recall_score,
    f1_score, confusion_matrix
)
from hsml.model_schema import ModelSchema
from hsml.schema import Schema
import pandas as pd
import os
import joblib
import seaborn as sns

mr = project.get_model_registry()

def _dtype_to_hsml(dtype) -> str:
    if pd.api.types.is_integer_dtype(dtype):
        return "int32"
    if pd.api.types.is_float_dtype(dtype):
        return "float32"
    if pd.api.types.is_bool_dtype(dtype):
        return "boolean"
    return "string"


inputs = [
    {
        "name": col,
        "type": _dtype_to_hsml(X_train[col].dtype),
        "description": col,
    }
    for col in X_train.columns
]
input_schema = Schema(inputs)

outputs = [
    {
        "type": "float32",
        "shape": [1],
        "description": "Probability of Ap >= threshold",
    }
]
output_schema = Schema(outputs)

model_schema = ModelSchema(
    input_schema=input_schema,
    output_schema=output_schema,
)

for h in range(1, MAX_HORIZON + 1):
    print(f"\ndYs? Training model for t+{h}")

    y_train = y_targets[h].iloc[train_mask]
    y_test  = y_targets[h].iloc[test_mask]

    # Safety checks
    assert len(X_train) == len(y_train)
    assert len(X_test) == len(y_test)
    assert X_train.index.max() < X_test.index.min()

    pos = y_train.sum()
    neg = len(y_train) - pos
    scale_pos_weight = (neg / max(pos, 1))

    model = XGBClassifier(
        n_estimators=300,
        max_depth=4,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        eval_metric="auc",
        random_state=42,
        scale_pos_weight=scale_pos_weight
    )

    model.fit(X_train, y_train)

    y_proba = model.predict_proba(X_test)[:, 1]
    y_pred  = (y_proba >= 0.5).astype(int)

    metrics = {
        "roc_auc": float(roc_auc_score(y_test, y_proba)),
        "accuracy": float(accuracy_score(y_test, y_pred)),
        "precision": float(precision_score(y_test, y_pred, zero_division=0)),
        "recall": float(recall_score(y_test, y_pred, zero_division=0)),
        "f1": float(f1_score(y_test, y_pred, zero_division=0)),
        "horizon_days": h,
        "ap_threshold": AP_THRESHOLD,
        "feature_view_version": FEATURE_VIEW_VERSION,
        "train_pos_class" : int(pos),
        "train_neg_class" : int(neg),
        "train_pos_ratio" : float(pos / (pos + neg))
    }

    cm = confusion_matrix(y_test, y_pred)
    print("Confusion matrix:")
    print(cm)
    print("Metrics:", metrics)

    # Save Model
    model_dir = f"aurora_model_h{h}"
    model_img_dir = model_dir + "/images"
    os.makedirs(model_dir, exist_ok=True)
    os.makedirs(model_img_dir, exist_ok=True)

    # Creating confusion matrix
    plt.figure(figsize=(4, 3))
    sns.heatmap(
        cm,
        annot=True,
        fmt="d",
        cmap="Blues",
        cbar=False
    )
    plt.xlabel("Predicted label")
    plt.ylabel("True label")
    plt.title(f"Confusion Matrix (t+{h})")

    plt.tight_layout()
    plt.savefig(f"{model_img_dir}/confusion_matrix_tplus{h}.png")
    plt.close()

    feature_importance_plot(model, model_img_dir)

    model_path = os.path.join(model_dir, "model.pkl")
    joblib.dump(model, model_path)
    

    model_name = f"{MODEL_NAME}_h{h}"
    model_description = (
        f"Binary aurora classifier (Ap >= 15) predicting t+{h} days ahead "
    )

    registered_model = mr.python.create_model(
        name= model_name,
        description= model_description,
        metrics=metrics,
        model_schema=model_schema,
        feature_view=feature_view,
        training_dataset_version=1
    )

    registered_model.save(model_dir)

    print(f"?. Model {model_name} saved and registered")



dYs? Training model for t+1
Confusion matrix:
[[227  67]
 [ 56  89]]
Metrics: {'roc_auc': 0.7619516772226131, 'accuracy': 0.7198177676537585, 'precision': 0.5705128205128205, 'recall': 0.6137931034482759, 'f1': 0.5913621262458472, 'horizon_days': 1, 'ap_threshold': 15, 'feature_view_version': 1, 'train_pos_class': 249, 'train_neg_class': 1503, 'train_pos_ratio': 0.1421232876712329}


  0%|          | 0/6 [00:00<?, ?it/s]

Uploading C:\Users\lppap\Documents\master\scalable_ML\id2223-project\notebooks\aurora\aurora_model_h1/feature_…

Uploading C:\Users\lppap\Documents\master\scalable_ML\id2223-project\notebooks\aurora\aurora_model_h1/model.pk…

Uploading C:\Users\lppap\Documents\master\scalable_ML\id2223-project\notebooks\aurora\aurora_model_h1\images/c…

...

Uploading C:\Users\lppap\Documents\master\scalable_ML\id2223-project\notebooks\aurora\aurora_model_h5\images/c…

Uploading C:\Users\lppap\Documents\master\scalable_ML\id2223-project\notebooks\aurora\aurora_model_h5\images/f…

Uploading C:\Users\lppap\Documents\master\scalable_ML\id2223-project\notebooks\aurora\aurora_model_h5\images/m…

Uploading C:\Users\lppap\Documents\master\scalable_ML\id2223-project\notebooks\aurora\model_schema.json: 0.000…

Model created, explore it at https://c.app.hopsworks.ai:443/p/1279154/models/aurora_xgboost_h5/4
?. Model aurora_xgboost_h5 saved and registered
